# Camera Calibration

from ..\CarND-Project4\CarND-Advanced-Lane-Lines-master\camera_cal  read in chessboard images and find corners

In [1]:
#code segment adapted from code presented in OpenCV-python-tutorials
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# path name for folder storing camera calibration images
pathname = 'C:\\Users\\meile\\Projects\\CarND-Project4\\CarND-Advanced-Lane-Lines-master\\camera_cal\\'

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
nx = 9
ny = 6

objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob(pathname+'*.jpg')

def corners_unwarp(images, nx, ny):

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        img_size = gray.shape[::-1]
    
        # step 1: capture the possible chess board corners using cv2.findChessboardCorners function
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

        # step 2: check to see if any corners are found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)

            corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            imgpoints.append(corners2)

            # Draw and display the corners on the image used to search for the presence of corners
            img = cv2.drawChessboardCorners(img, (nx,ny), corners2,ret)
            save_name = fname + "copy.jpg"  #<-- this is a dirty workaround for image renaming.
            
            cv2.imwrite(save_name,img)

            #also taken from Open-CV-python tutorials, same as what is used in the Udacity class materials
            #Processing of step 2.  
            
            #So now we have our object points and image points we are ready to go for calibration. 
            #For that we use the function, cv2.calibrateCamera(). It returns the camera matrix, distortion coefficients, 
            #rotation and translation vectors etc.
            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

            h,  w = img.shape[:2]
            newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
        
            undist = cv2.undistort(img, newcameramtx, dist, None, mtx)
        
            src = np.float32([corners[0],corners[nx-1],corners[-1],corners[-nx]])
            offset = 100
            # define 4 destination points dst = np.float32([[,],[,],[,],[,]])
            dst = np.float32([[offset, offset],[img_size[0]-offset,offset],[img_size[0]-offset,img_size[1]-offset],[offset, img_size[1] - offset]])
            # use cv2.getPerspectiveTransform() to get M, the transform matrix
            M = cv2.getPerspectiveTransform(src, dst)
            Minv = cv2.getPerspectiveTransform(dst, src)
            # use cv2.warpPerspective() to warp your image to a top-down view
            warped_input = cv2.warpPerspective(undist, M, img_size, flags=cv2.INTER_LINEAR)

    return warped_input, M, newcameramtx, dist, mtx

#unwraping the image, into a warped image.  All variables are renamed to identify the camera related images, not the road images
warped_cam, M_cam, newcameramtx_cam, dist_cam, mtx_cam= corners_unwarp(images, nx, ny)

save_name_warped = pathname+'warped_cam_img.jpg'

cv2.imwrite(save_name_warped, warped_cam)

True

# Pipeline (test images)

# # undistorting test images

# # # Provide an example of a distortion-corrected image.

In [2]:
#### Undistorting Test images ####

def correct_dist(fname, newcameramtx, dist, mtx):
    img2 = cv2.imread(fname)
    gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

    undist2 = cv2.undistort(img2, newcameramtx, dist, None, mtx)

    return img2, undist2

def correct_dist_video(img2, newcameramtx, dist, mtx):
    
    gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

    undist2 = cv2.undistort(img2, newcameramtx, dist, None, mtx)

    return img2, undist2


# # # Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image. Provide an example of a binary image result.

In [3]:
cv2.destroyAllWindows()


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.close('all') # Close a figure window

sobel_kernel=3
#mag_thresh=(30, 255)
s_thresh=(220, 255) #(170, 255) 
sx_thresh=(30,100)#(20, 100)

def pipeline(img2, s_thresh, sx_thresh):

    # from course material.  Apply the following steps to img2
    # 1) Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img2, cv2.COLOR_RGB2HSV).astype(np.float)
    v_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    
    # 2) Take the gradient in x + y
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    sobely = cv2.Sobel(s_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    sobelx_glare = cv2.Sobel(v_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    sobely_glare = cv2.Sobel(v_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # 3) Calculate the magnitude 
    sqrt_sobelx = np.sqrt(sobelx**2)
    sqrt_sobely = np.sqrt(sobely**2)
    abs_sobelx = np.absolute(sqrt_sobelx)
    abs_sobely = np.absolute(sqrt_sobely)
    
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobelx/(2*np.max(abs_sobelx)))+ np.uint8(255*abs_sobely/(2*np.max(abs_sobely)))
    scaled_s_channel = np.uint8(255*s_channel/np.max(s_channel))
        
    # 5) Create a binary mask where mag thresholds are met

    # Threshold x gradient
    #binary_output -->sxbinary
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    color_binary[(color_binary >=1)] = 255

    # 6) Return this mask as your binary_output image

    return color_binary

In [4]:
def bird_view_transform(undist_road_view, corners):         
    #undist is an image that has been previously undistorted using the computed coefs of the camera lense
    src = corners
    offset = 100
    #define 4 destination points dst = np.float32([[,],[,],[,],[,]])
    dst = np.float32([[offset, offset],[img_size[0]-offset,offset],[img_size[0]-offset,img_size[1]-offset],[offset, img_size[1] - offset]])
    # use cv2.getPerspectiveTransform() to get M, the transform matrix and Minv, the inverse transform matrix.
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    # use cv2.warpPerspective() to warp your image to a top-down view
    warped_to_birdview = cv2.warpPerspective(undist_road_view, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped_to_birdview, M, Minv

In [5]:
def road_view_transform(warped_birdview, Minv_road, img_size):
    #basic reverse function, may need tuning
    
    warped_to_roadview = cv2.warpPerspective(warped_birdview, Minv_road, img_size, flags=cv2.INTER_LINEAR)
    
    return warped_to_roadview

In [6]:
#functions copied from Udacity class exercise material#

def window_mask(width, height, birdview_img_ref, center,level):
    output_masked = np.zeros_like(birdview_img_ref)
    output_masked[int(birdview_img_ref.shape[0]-(level+1)*height):int(birdview_img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),birdview_img_ref.shape[1])] = 1
    return output_masked

In [7]:
def find_window_centroids(image_to_search, window_width, window_height, margin,prev_lx,prev_rx):
    
    window_centroids_found = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    search_window_width = window_width
    #check if lines have been previously found, and define x range to search for left line and right line
    if (prev_lx < search_window_width):# to consider that no previous left lane location is known
        # Sum quarter bottom of image to get slice, could use a different ratio
        l_sum = np.sum(image_to_search[int(3*image_to_search.shape[0]/4):,:int(image_to_search.shape[1]/2)], axis=0)
        #l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    else:
        l_sum_xmin = int(prev_lx - search_window_width)
        l_sum_xmax = int(prev_lx + search_window_width)
        l_sum = np.sum(image_to_search[l_sum_xmin:l_sum_xmax,:int(image_to_search.shape[1]-window_height)], axis=0)
    
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
        
    if ((prev_rx < int(image_to_search.shape[1]/2))or(prev_rx > int(image_to_search.shape[1]-search_window_width-1))):
        #the minus 1 at the end is because the indexes of the image go from 0 to image.size[x]-1, and here 
        #the risk is to try performing operations using pixels that are out of range of the image to search
        #Sum quarter bottom right hand side of the image, considering no right lane line was detected previously
        r_sum = np.sum(image_to_search[int(3*image_to_search.shape[0]/4):,int(image_to_search.shape[1]/2):], axis=0)
        
    else:
        r_sum_xmin = int(prev_rx - search_window_width)
        r_sum_xmax = int(prev_rx + search_window_width)
        r_sum = np.sum(image_to_search[r_sum_xmin:r_sum_xmax,:int(image_to_search.shape[1]-window_height)], axis=0)
        
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image_to_search.shape[1]/2)
                                         
    # Sum quarter bottom of image to get slice, could use a different ratio //this is the original code snippet, without any additional tracking of previous lane positions
    #l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
    #l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    #r_sum = np.sum(image_to_search[int(3*image_to_search.shape[0]/4):,int(image_to_search.shape[1]/2):], axis=0)
    #r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image_to_search.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids_found.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image_to_search.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(image_to_search[int(image_to_search.shape[0]-(level+1)*window_height):int(image_to_search.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,image_to_search.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,image_to_search.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # Add what we found for that layer
        window_centroids_found.append((l_center,r_center))

    return window_centroids_found

In [8]:
def poly_image_extractor(window_centroids,image, level):
    
    img_height = image.shape[0]
    img_width = image.shape[1]
    # Create empty lists to receive left and right lane pixel indices for x and y axes
    left_lane_inds = []
    right_lane_inds = []
    leftx=[]
    lefty=[]
    rightx=[]
    righty=[]
    leftyx=[]
    rightyx=[]

    for level in range((int)(len(window_centroids))):
        
        if (window_centroids[level][0]!=0):
            lx=int(window_centroids[level][0])
            ly=int((img_height-level*80-40))
            if ((lx <=img_width)&(ly<=img_height)):
                left_lane_inds.append(lx)
                leftx.append(lx) #appending here the pixel in image width
                lefty.append(ly) #appending here the pixel in image height, computed from the image height minus the window level and taken at the center of the window (50pix wide, 80 high)
                leftyx.append((ly,lx))
                
        if (window_centroids[level][1]!=0):
            rx = int(window_centroids[level][1])
            ry=int(img_height-level*80-40)
            if ((rx <=img_width)&(ry<=img_height)):
                right_lane_inds.append(rx)
                rightx.append(rx) #appending here the pixel in image width
                righty.append(ry) #appending here the pixel in image height, computed from the image height minus the window level and taken at the center of the window (50pix wide, 80 high)
                rightyx.append((ry, rx))

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    
    #this is a temporary fix to getting array indices out of range (and non-plottable to the image, without error)
    #I would like to simply "erase" the index to this section of the fitted polynom
    for ix in range((int)(len(left_fitx))):
        if (left_fitx[ix] < 0 ):
            left_fitx[ix]= left_fit[0]*(0.5*img_width)**2+left_fit[1]*(0.5*img_width)+left_fit[1]
            
        
    for ix in range((int)(len(right_fitx))):
        if(right_fitx[ix] > img_width):
            right_fitx[ix] = right_fit[0]*(0.5*img_width)**2+right_fit[1]*(0.5*img_width)+right_fit[1]

    ploty.astype(int)
    ploty_int = ploty.astype(int)
    left_fitx_int = left_fitx.astype(int)
    right_fitx_int = right_fitx.astype(int)

    # Create an image to draw on and an image to show the selection window
    polyfit_img = np.zeros_like(image)
    color_polyfit_img = np.dstack((polyfit_img, polyfit_img, polyfit_img))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx_int, ploty_int]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx_int, ploty_int])))])
    pts = np.hstack((pts_left, pts_right))
    pts_int =pts.astype(int)

    # Draw the lane onto the warped blank image
    #cv2.fillPoly(color_polyfit_img, pts_int, (0,255, 0))
    
    # Color in left and right line pixels
    #polyfit_img[(lefty,leftx)] = [255, 255, 255]
    #polyfit_img[(righty,rightx)] = [255, 255, 255]
    polyfit_img[(ploty_int,left_fitx_int)] = [255,0,255]
    polyfit_img[(ploty_int,right_fitx_int)] = [255,0,0]
    #polyfit_img = cv2.addWeighted(color_polyfit_img, 1, image, 0.3, 0)
    #print ('poly fit end')
    
    #image_to_search[int(3*image_to_search.shape[0]/4):,:int(image_to_search.shape[1]/2)]
    
    #polyfit_img2= window_mask(width=10, height=10, polyfit_img, center=left_fitx_int,level=nbstrips)
    #polyfit_img2= window_mask(width=10, height=10, polyfit_img, center=right_fitx_int,level=nbstrips)
       
    #polyfit_img2= window_mask(10, 10, polyfit_img, left_fitx_int,nb_strips)
    #polyfit_img2= window_mask(10, 10, polyfit_img, right_fitx_int,nb_strips)
 
    #in this section of the code, we will compute the radius of the curvature, answer given in meters.
    #code taken from course material
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = (float)(3/56) # (30/720) meters per pixel in y dimension
    xm_per_pix = (float)(3.7/676) # (3.7/700) meters per pixel in x dimension
    lefty_ws = []
    leftx_ws = []
    righty_ws = []
    rightx_ws = []
    
    # Fit new polynomials to x,y in world space
    # Fit a second order polynomial to each, in world space
    for inx in range(0,len(lefty)):
        lefty_ws.append(lefty[inx]*ym_per_pix)
    
    for inx in range(0,len(leftx)):
        leftx_ws.append(leftx[inx]*xm_per_pix)
    
    for inx in range(0,len(righty)):
        righty_ws.append(righty[inx]*ym_per_pix)
    
    for inx in range(0,len(rightx)):
        rightx_ws.append(rightx[inx]*xm_per_pix)
    
    left_fit_cr = np.polyfit(lefty_ws, leftx_ws, 2)
    right_fit_cr = np.polyfit(righty_ws, rightx_ws, 2)

    # Calculate the new radii of curvature
    y_eval = np.max(ploty_int)
    y_eval_m =y_eval*ym_per_pix # evaluation of y near the car (bottom of the picture)

    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    left_curverad_m = ((1 + (2*left_fit_cr[0]*y_eval_m + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad_m = ((1 + (2*right_fit_cr[0]*y_eval_m + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    
    #print('left lane estimate poly in pixels:' + str(round(left_fit[0],6)) + '*x^2 +' + str(round(left_fit[1],6)) + '*x +'+ str(round(left_fit[2],6)))
    #print('right lane estimate poly in pixels:' + str(round(right_fit[0],6)) + '*x^2 +' + str(round(right_fit[1],6)) + '*x +'+ str(round(right_fit[2],6)))
    
    #print('left lane estimate poly in meters:' + str(round(left_fit[0]*ym_per_pix,6)) + '*x^2 +' + str(round(left_fit[1],6)) + '*x +'+ str(round(left_fit[2],2)))
    #print('right lane estimate poly in meters:' + str(round(right_fit[0]*ym_per_pix,6)) + '*x^2 +' + str(round(right_fit[1],6)) + '*x +'+ str(round(right_fit[2],2)))

    
    #print('evaluated curvature : \n',left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m
    
    # Calculate the position of the car with respect to the center of the lane:
    # max index
    left_lane_pos = left_fitx_int[(len(left_fitx_int) - 1)]
    right_lane_pos = right_fitx_int[(len(right_fitx_int) - 1)]
    
    #not robust here, since it can fail if either right lane or left lane is not found in the current image
    lane_width_pix = right_lane_pos - left_lane_pos
    lane_width_m = lane_width_pix * xm_per_pix
    mid_lane_pos = int(lane_width_pix/2)
    car_pos = int(img_width/2)-mid_lane_pos
    car_pos_m = car_pos*xm_per_pix
    
    #print('lane width in pixels : ', round(lane_width_pix,2))
    #print('position of the middle of the lane in pixels : ', round(mid_lane_pos,2))
    #print('position of the car with respect to the lane in pixels : ', round(car_pos,2))
    #print('lane width in meters : ', round(lane_width_m,2))
    #print('position of the car with respect to the lane center in meters : ', round(car_pos_m,2))
    
    text_print = '\n'
    #text_print += 'left lane estimate poly in meters:' + str(round(left_fit[0]*ym_per_pix,6)) + '*x^2 +' + str(round(left_fit[1],6)) + '*x +'+ str(round(left_fit[2],2))+'\n'
    #text_print +='right lane estimate poly in meters:' + str(round(right_fit[0]*ym_per_pix,6)) + '*x^2 +' + str(round(right_fit[1],6)) + '*x +'+ str(round(right_fit[2],2))+'\n'
    text_print += 'evaluated curvature left: '+str(left_curverad)+ 'm \nevaluated curvature right: '+ str(right_curverad)+ 'm'
    text_print += 'lane width in meters : '+ str(round(lane_width_m,2)) +'\n'
    text_print += 'position of the car with respect to the lane center in meters : '+ str(round(car_pos_m,2))+ '\n'
    
    text_print=(''.join(['Curvature Radius left = ', str(left_curverad), ' meters']))
    
    return ploty_int,left_fit, right_fit, polyfit_img,text_print #result (replaced by out_img)
    

In [9]:
#functions copied from Udacity class exercise material#

def template_creator(window_centroids, warped_image, window_width, window_height, margin):
    # If we found any window centers
    if len(window_centroids) > 0:
    
        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped_image)
        r_points = np.zeros_like(warped_image)
        
        #print('window_centroids[0][0]: ',window_centroids[0][0])
        #print('window_centroids[0][1]: ',window_centroids[0][1])
        
        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped_image,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped_image,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template_r = np.array(r_points,np.uint8) # np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        template_l = np.array(l_points,np.uint8)
        zero_channel = np.zeros_like(template_r) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template_r,template_l)),np.uint8) # make window pixels green for right pixels
        warpage = np.array(cv2.merge((warped_image,warped_image,warped_image)),np.uint8) # making the original road pixels 3 color channels
        warped_template = cv2.addWeighted(warpage, 1.5, template, 0.5, 0.0) # overlay the orignal road image with window results
        
        level=len(window_centroids)
        ploty_fit, left_fit, right_fit, resulting_polyfit_template, text_to_print = poly_image_extractor(window_centroids, warped_template, level) 
        #resulting_template[lefty,leftx,:] = [255, 0, 0]
        #resulting_template[righty,rightx,:] = [0, 0, 255]
        #plt.imshow(resulting_template)
        #plt.plot(left_fit, ploty, color='yellow')
      
        
    # If no window centers found, just display orginal road image
    else:
        resulting_polyfit_template = np.array(cv2.merge((warped_image,warped_image,warped_image)),np.uint8)
    
    return template, warpage, resulting_polyfit_template, text_to_print


In [10]:
#test the functions proivded above

#path name for test images
pathname2 = 'C:\\Users\\meile\\Projects\\CarND-Project4\\CarND-Advanced-Lane-Lines-master\\test_images\\'

test_images = glob.glob(pathname2 + '*.jpg')


#these are the corners identified in the picture to perform the bird view transform (width, height), 
#starting with the top right corner, top left corner, bottom left corner, bottom right corner

corners = np.float32(
    [[585,450],
     [715,450],
     [1210,600],
     [360,600]])

#parameters for defining the window search are set here
window_width = 50 
window_height = 80 # Break image into 9 vertical layers since image height is 720
margin = 100 # How much to slide left and right for searching


for fname in test_images:
    # using the camera matrix corrections saved in the object newcameramtx_cam to undistort the image fname.
    # original_road_image : original image before correcting for the camera lense distortion
    # undistorted_road_image : image after correcting for distortion
    # pipelined_road_binary : the image result from applying soebel functions to search for gradients in the undistorted_road_image
    # warped_road_image : sets the pipelined_road_image to a binary image (0 or 255) by adding the pixels from each color channel of the pipelined_road_image together
    # warped_road_image_birdview : this is the resulting image of perspective transform of the warped_road_image. It is the image used to search for window centroids
    # template_created : this is a mask that plots where window centroids were found in the warped_road_image_birdview
    
    original_road_image, undistorted_road_image = correct_dist(fname, newcameramtx_cam, dist_cam, mtx_cam) 
    gray = cv2.cvtColor(undistorted_road_image,cv2.COLOR_BGR2GRAY)
    img_size = gray.shape[::-1]
    
    pipelined_road_binary = pipeline(undistorted_road_image, s_thresh, sx_thresh)
    
    ###this hereafter needs to be changed to improve my algorithme:####
    warped_road_image=pipelined_road_binary[:,:,0]+pipelined_road_binary[:,:,1]+pipelined_road_binary[:,:,2]
    warped_road_image_birdview, M_road, Minv_road = bird_view_transform(warped_road_image, corners)
    #print(warped2.shape)
    previous_lx = 0#820#162
    previous_rx = 0#160#824
    window_centroids = find_window_centroids(warped_road_image_birdview, window_width, window_height, margin, previous_lx, previous_rx)
    template_created, warpage_created, resulting_polyfit_template_created, resulting_text_to_print = template_creator(window_centroids, warped_road_image_birdview, window_width, window_height, margin)
    warped_to_roadview_template = road_view_transform(resulting_polyfit_template_created, Minv_road, img_size)
    
    undistorted_road_image_lined = cv2.addWeighted(undistorted_road_image, 1.0, warped_to_roadview_template, 10.5, 0.0)
    print('####',fname,'####')
    print(resulting_text_to_print)
    print('################')
    
    
    # Display the final results
    

    #cv2.imshow('original_road_image',original_road_image)
    #cv2.imshow('undistorted_road_image',undistorted_road_image)
    #cv2.imshow('pipelined_road_binary',pipelined_road_binary)
    #cv2.imshow('warped_road_image',warped_road_image)
    #cv2.imshow('warped_road_image_birdview',warped_road_image_birdview)
    #cv2.imshow('template_created',template_created)
    #cv2.imshow('warpage_created',warpage_created)
    #cv2.imshow('resulting_polyfit_template_created',resulting_polyfit_template_created)
    #cv2.imshow('warped_to_roadview_template',warped_to_roadview_template)
    #cv2.imshow('undistorted_road_image_lined',undistorted_road_image_lined)
    #cv2.waitKey(500)
    
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    save_name = fname + 'original_road_image.jpg'
    cv2.imwrite(save_name,original_road_image)
    save_name = fname + 'undistorted_road_image.jpg'
    cv2.imwrite(save_name,undistorted_road_image)
    save_name = fname + 'pipelined_road_binary.jpg'
    cv2.imwrite(save_name,pipelined_road_binary)
    save_name = fname + 'warped_road_imagepipelined_road_binary.jpg'
    cv2.imwrite(save_name,warped_road_image)
    save_name = fname + 'warped_road_image_birdview.jpg'
    cv2.imwrite(save_name,warped_road_image_birdview)
    save_name = fname + 'template_created.jpg'
    cv2.imwrite(save_name,template_created)
    save_name = fname + 'warpage_created.jpg'
    cv2.imwrite(save_name,warpage_created)
    save_name = fname + 'resulting_polyfit_template_created.jpg'
    cv2.imwrite(save_name,resulting_polyfit_template_created)
    save_name = fname + 'warped_to_roadview_template.jpg'
    cv2.imwrite(save_name,warped_to_roadview_template)
    save_name = fname + 'undistorted_road_image_lined.jpg'
    cv2.imwrite(save_name,undistorted_road_image_lined)
    
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(undistorted_road_image_lined,resulting_text_to_print,(10,500), font, 0.6,(255,255,255),2,cv2.LINE_AA)
    #cvInitFont(CvFont* font, int font_face, double hscale, double vscale, double shear=0, int thickness=1, int line_type=8 )
    #cv2.putText(undistorted_road_image_lined,resulting_text_to_print,hscale=0.5)
    
    save_name = fname + "undistorted_road_image_lined_commented.jpg"  #<-- this is a dirty workaround for image renaming.            
    cv2.imwrite(save_name,undistorted_road_image_lined)


    #print('-----------------------------------')

#### C:\Users\meile\Projects\CarND-Project4\CarND-Advanced-Lane-Lines-master\test_images\test3.jpg ####
Curvature Radius left = 685.576344119 meters
################


This section provides the code for video editing

In [11]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

video_path='C:\\Users\\meile\\Projects\\CarND-Project4\\CarND-Advanced-Lane-Lines-master\\'

In [12]:
def process_image(video_image):
    corners = np.float32(
    [[585,450],
     [715,450],
     [1210,600],
     [360,600]])

    #parameters for defining the window search are set here
    window_width = 50 
    window_height = 80 # Break image into 9 vertical layers since image height is 720
    margin = 100 # How much to slide left and right for searching

    # using the camera matrix corrections saved in the object newcameramtx_cam to undistort the image fname.
    # original_road_image : original image before correcting for the camera lense distortion
    # undistorted_road_image : image after correcting for distortion
    # pipelined_road_binary : the image result from applying soebel functions to search for gradients in the undistorted_road_image
    # warped_road_image : sets the pipelined_road_image to a binary image (0 or 255) by adding the pixels from each color channel of the pipelined_road_image together
    # warped_road_image_birdview : this is the resulting image of perspective transform of the warped_road_image. It is the image used to search for window centroids
    # template_created : this is a mask that plots where window centroids were found in the warped_road_image_birdview
    
    original_road_image, undistorted_road_image = correct_dist_video(video_image, newcameramtx_cam, dist_cam, mtx_cam) 
    gray = cv2.cvtColor(undistorted_road_image,cv2.COLOR_BGR2GRAY)
    img_size = gray.shape[::-1]
    
    pipelined_road_binary = pipeline(undistorted_road_image, s_thresh, sx_thresh)
    
    ###this hereafter needs to be changed to improve my algorithme:####
    warped_road_image=pipelined_road_binary[:,:,0]+pipelined_road_binary[:,:,1]+pipelined_road_binary[:,:,2]
    warped_road_image_birdview, M_road, Minv_road = bird_view_transform(warped_road_image, corners)
    #no automatic search function is implemented here, because the search filter did not provide satisfactory results
    previous_lx = 0
    previous_rx = 0
    window_centroids = find_window_centroids(warped_road_image_birdview, window_width, window_height, margin, previous_lx, previous_rx)
    template_created, warpage_created, resulting_polyfit_template_created, resulting_text_to_print = template_creator(window_centroids, warped_road_image_birdview, window_width, window_height, margin)
    warped_to_roadview_template = road_view_transform(resulting_polyfit_template_created, Minv_road, img_size)
    
    undistorted_road_image_lined = cv2.addWeighted(undistorted_road_image, 1.0, warped_to_roadview_template, 10.5, 0.0)
    cv2.putText(img=np.copy(undistorted_road_image_lined), text=(''.join([resulting_text_to_print])), org=(100,100),fontFace=5, fontScale=1, color=(255,255,255), thickness=1)
    
    return undistorted_road_image_lined

In [13]:
white_output = video_path+'project_video_output.mp4'
white_input = video_path+'project_video.mp4'
print(white_output)
print(white_input)
clip1 = VideoFileClip(white_input)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
get_ipython().magic(u'time white_clip.write_videofile(white_output, audio=False)')

#video_clip.reader.close()
#video_clip.audio.reader.close_proc()

C:\Users\meile\Projects\CarND-Project4\CarND-Advanced-Lane-Lines-master\project_video_output.mp4
C:\Users\meile\Projects\CarND-Project4\CarND-Advanced-Lane-Lines-master\project_video.mp4
[MoviePy] >>>> Building video C:\Users\meile\Projects\CarND-Project4\CarND-Advanced-Lane-Lines-master\project_video_output.mp4
[MoviePy] Writing video C:\Users\meile\Projects\CarND-Project4\CarND-Advanced-Lane-Lines-master\project_video_output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [06:02<00:00,  3.51it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: C:\Users\meile\Projects\CarND-Project4\CarND-Advanced-Lane-Lines-master\project_video_output.mp4 

Wall time: 6min 3s


In [14]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))